In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import json

In [3]:
# Create the connection engine
engine = create_engine("sqlite:///Resources/strains.sqlite")
conn = engine.connect()

In [4]:
medical_effects = pd.read_sql('SELECT Strains."name" as strain_name, \
	 MedicalEffects."medical_effect" as "medical_effects", ROUND(AVG(MedicalEffects.rating),2) as medical_rating \
FROM Strains \
INNER JOIN MedicalEffects \
ON Strains.id=MedicalEffects.strain_id \
GROUP BY name',conn)

In [5]:
medical_effects

,strain_name,medical_effects,medical_rating
0,00 Kush,Autoimmune Diseases and Inflammation,5.00
1,1024,Pain,4.00
2,2046,Epilepsy,4.50
3,4 Best 4 You,Psychiatric Symptoms,3.00
4,5G's Red x Sour Diesel,Nausea and Vomiting,5.00
...,...,...,...
266,Wild Thailand,Nausea and Vomiting,2.00
267,X-Dog,Pain,4.00
268,X-Kush #2,Nausea and Vomiting,5.00
269,Y Griega,Pain,2.63


In [6]:
strains = pd.read_sql("SELECT id AS strain_id, name as strain_name, breeder as breeder_id FROM Strains",conn)

In [7]:
strains.head()

,strain_id,strain_name,breeder_id
0,1,Af-Pak,1
1,2,00 Cheese,2
2,3,Alien BubbleGum,3
3,4,Cherry OG Kush,4
4,5,Ak 420,5


In [8]:
breeders = pd.read_sql("SELECT id as breeder_id, name as breeder_name FROM Breeders", conn)

In [9]:
breeders.head()

,breeder_id,breeder_name
0,1,210Beans
1,2,00 Seeds Bank
2,3,207 Seeds
3,4,420 Seeds
4,5,420 Genetics


In [10]:
# join strains with breeders on breeder id
strains_breeders = pd.merge(strains,breeders,how='inner',on='breeder_id')

In [11]:
strains_breeders.head()

,strain_id,strain_name,breeder_id,breeder_name
0,1,Af-Pak,1,210Beans
1,8,BlackberryKush,1,210Beans
2,13,Pak-Man,1,210Beans
3,2,00 Cheese,2,00 Seeds Bank
4,12,00 Kush,2,00 Seeds Bank


In [12]:
#Number of strains produced by more than 1 breeder
num_dups = len(strains_breeders.strain_name) - strains_breeders.strain_name.nunique()
num_dups

1672

In [14]:
#medical_effects = pd.read_sql("SELECT strain_id, name as medical_effect, info as medical_info, rating as medical_rating FROM MedicalEffects", conn)
#medical_effects
#medical_effects = pd.read_sql("SELECT id as name_id, name as medical_effect, info as medical_info, rating as medical_rating FROM MedicalEffects", conn)

In [15]:
file = 'Resources/cannabis.csv'
cannabis_df = pd.read_csv(file)

In [16]:
cannabis_cols = ["name", "type", "thc_level", "most_common_terpene", "relaxed", "happy", "euphoric", "uplifted"]
cannabis_transformed = cannabis_df[cannabis_cols].copy()
#cannabis_transformed

In [17]:
cannabis_transformed = cannabis_transformed.rename(columns={"name": "Name","type": "Type","thc_level": "THC_Level","most_common_terpene": "Terpene",
                                                           "relaxed": "Relaxed", "happy": "Happy", "euphoric": "Euphoric", "uplifted": "Uplifted"
                                                           })
#cannabis_transformed

In [18]:
cannabis_transformed.drop_duplicates("Name", inplace=True)

In [19]:
cannabis_transformed = cannabis_transformed.reset_index()

In [20]:
cannabis_transformed=cannabis_transformed.rename(columns={'index':'id'})

In [21]:
cannabis_transformed['Name']=cannabis_transformed['Name'].str.replace('-',' ')

In [36]:
csv_strains = cannabis_transformed[['Name']]

In [40]:
#strains_combined = pd.merge(sql_strains,csv_strains, on='Name',how='inner').drop_duplicates()

In [22]:
csv_strains = cannabis_transformed[['Name']]
csv_effects = cannabis_transformed[['Name','Type']]
csv_ratings = cannabis_transformed[['Name','THC_Level']]

In [23]:
# first find the strain overlaps between 2 data sources
sql_strains = strains.drop(['strain_id','breeder_id'],axis=1)

In [29]:
#strains_combined = pd.merge(sql_strains,csv_strains, on='strain_name',how='inner').drop_duplicates()
#strains_combined

In [42]:
file = "Resources/leafly_strain_data.json"

In [43]:
strains_df = pd.read_json('Resources/leafly_strain_data.json')
strains_df

,name,img_url,type,thc_level,most_common_terpene,description,effects
0,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...","{'relaxed': '66%', 'happy': '54%', 'euphoric':..."
1,Wedding Cake,https://images.leafly.com/flower-images/weddin...,Hybrid,22%,Limonene,Wedding Cake is a potent indica-hybrid marijua...,"{'relaxed': '60%', 'happy': '50%', 'euphoric':..."
2,Runtz,https://images.leafly.com/flower-images/runtz-...,Hybrid,19%,Limonene,Runtz is a rare hybrid marijuana strain by Coo...,"{'happy': '62%', 'euphoric': '48%', 'relaxed':..."
3,Gelato,https://images.leafly.com/flower-images/gelato...,Hybrid,17%,Caryophyllene,"Gelato, aka Larry Bird, is a hybrid marijuana ...","{'relaxed': '54%', 'happy': '49%', 'euphoric':..."
4,Blue Dream,https://images.leafly.com/flower-images/blue-d...,Hybrid,18%,Myrcene,Blue Dream is a sativa-dominant hybrid marijua...,"{'happy': '57%', 'euphoric': '52%', 'relaxed':..."
...,...,...,...,...,...,...,...
4757,Khyber Kush,,Hybrid,25%,Myrcene,An ode to the Khyber Pass that bridges Pakista...,{}
4758,Dorit,,Indica,,,Dorit is a 70/30 indica-dominant strain tailor...,"{'relaxed': '83%', 'euphoric': '66%', 'uplifte..."
4759,Berry Noir,,Indica,,,Berry Noir is an indica strain that competed i...,"{'relaxed': '66%', 'sleepy': '50%', 'euphoric'..."
4760,South Central LA,,Indica,,,South Central LA is a high-THC indica strain f...,"{'uplifted': '75%', 'relaxed': '75%', 'hungry'..."


In [44]:
df1test=strains_df.effects.dropna().apply(pd.Series)
df1test

,relaxed,happy,euphoric,uplifted,sleepy,dry_mouth,dry_eyes,dizzy,paranoid,anxious,...,fibromyalgia,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus
0,66%,54%,51%,39%,27%,24%,12%,5%,4%,4%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60%,50%,41%,35%,NaN,17%,7%,4%,2%,2%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,47%,62%,48%,43%,NaN,9%,4%,3%,NaN,1%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54%,49%,46%,37%,NaN,20%,9%,4%,3%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,48%,57%,52%,48%,NaN,29%,16%,6%,6%,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4758,83%,NaN,66%,33%,16%,NaN,16%,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4759,66%,NaN,50%,16%,50%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4760,75%,NaN,75%,75%,NaN,75%,50%,NaN,25%,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
strains_df['effects'].dtypes

dtype('O')

In [48]:
strains_df['effects'].astype(str)

0       {'relaxed': '66%', 'happy': '54%', 'euphoric':...
1       {'relaxed': '60%', 'happy': '50%', 'euphoric':...
2       {'happy': '62%', 'euphoric': '48%', 'relaxed':...
3       {'relaxed': '54%', 'happy': '49%', 'euphoric':...
4       {'happy': '57%', 'euphoric': '52%', 'relaxed':...
                              ...                        
4757                                                   {}
4758    {'relaxed': '83%', 'euphoric': '66%', 'uplifte...
4759    {'relaxed': '66%', 'sleepy': '50%', 'euphoric'...
4760    {'uplifted': '75%', 'relaxed': '75%', 'hungry'...
4761    {'uplifted': '100%', 'relaxed': '100%', 'talka...
Name: effects, Length: 4762, dtype: object

In [49]:
# Adding two new columns to the existing dataframe.
# bydefault splitting is done on the basis of single space.
#strains_df[['relaxed','happy','euphoric','uplifted']] = strains_df.effects.str.split('', expand=True)
   
#strains_df

In [45]:
connection_string = "postgres:851990Teja$@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [46]:
# Confirm tables
engine.table_names()

[]

In [41]:
#starter code, must edit
#strain_Name.to_sql(name='Name', con=engine, if_exists='append', index=True)